# Checklist
[ ] Trim
[ ] Concat
[ ] Resize
[ ] add_title
[ ] chèn logo
[ ] render 

# Basics

In [6]:
import logging
import subprocess

# Tạo logger riêng cho module này
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)  # hoặc INFO nếu muốn ít log hơn

# Nếu chưa có handler nào, thêm console handler
if not logger.handlers:
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(logging.Formatter(
        '[%(levelname)s] %(asctime)s - %(name)s: %(message)s',
        datefmt='%H:%M:%S'
    ))
    logger.addHandler(console_handler)
FFMPEG_PATH = "C:/Users/ADMIN/AppData/Local/Microsoft/WinGet/Packages/Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe/ffmpeg-7.1.1-full_build/bin/ffmpeg.exe"
FFPROBE_PATH = "C:/Users/ADMIN/AppData/Local/Microsoft/WinGet/Packages/Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe/ffmpeg-7.1.1-full_build/bin/ffprobe.exe"

def runf(args: list):


    args[0] = FFMPEG_PATH  # đảm bảo gọi đúng ffmpeg.exe thực

    logger.info("Running FFmpeg: %s", " ".join(args))
    result = subprocess.run(args, capture_output=True, text=True)
    
    if result.returncode != 0:
        logger.error("FFmpeg failed:\n%s", result.stderr)
        raise RuntimeError("FFmpeg command failed.")
    
    logger.debug("FFmpeg output:\n%s", result.stdout)
    return result.stdout


# Trim video

In [7]:
def trim_video(input_path: str, start: float, end: float, output_path: str, reencode: bool = False):
    """
    Cắt video từ giây `start` đến `end` và lưu vào `output_path`.
    
    :param input_path: Đường dẫn video gốc
    :param start: Thời gian bắt đầu (giây)
    :param end: Thời gian kết thúc (giây)
    :param output_path: Nơi lưu video đã cắt
    :param reencode: Nếu True, sẽ mã hóa lại video (chính xác hơn)
    """
    if end <= start:
        raise ValueError("Giá trị 'end' phải lớn hơn 'start'.")

    duration = end - start

    if reencode:
        args = [
            "ffmpeg", "-y", "-i", input_path,
            "-ss", str(start),
            "-t", str(duration),
            "-c:v", "libx264",
            "-c:a", "aac",
            output_path
        ]
    else:
        args = [
            "ffmpeg", "-y",
            "-ss", str(start),
            "-to", str(end),
            "-i", input_path,
            "-c", "copy",
            output_path
        ]

    logger.info("Trim video from %.2fs to %.2fs (%s → %s)", start, end, input_path, output_path)
    runf(args)


In [8]:
trim_video(
    input_path="D:/ThienPV/code/demo/data/TestAPI.mp4",
    start=10.95,
    end=14.469,
    output_path="D:/ThienPV/code/demo/.trash/TestAPI1.mp4",
    reencode=True
)
trim_video(
    input_path="D:/ThienPV/code/demo/data/TestAPI.mp4",
    start=0,
    end=10.95,
    output_path="D:/ThienPV/code/demo/.trash/TestAPI0.mp4",
    reencode=True
)


[INFO] 17:33:22 - __main__: Trim video from 10.95s to 14.47s (D:/ThienPV/code/demo/data/TestAPI.mp4 → D:/ThienPV/code/demo/.trash/TestAPI1.mp4)
[INFO] 17:33:22 - __main__: Running FFmpeg: C:/Users/ADMIN/AppData/Local/Microsoft/WinGet/Packages/Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe/ffmpeg-7.1.1-full_build/bin/ffmpeg.exe -y -i D:/ThienPV/code/demo/data/TestAPI.mp4 -ss 10.95 -t 3.519 -c:v libx264 -c:a aac D:/ThienPV/code/demo/.trash/TestAPI1.mp4
[INFO] 17:33:23 - __main__: Trim video from 0.00s to 10.95s (D:/ThienPV/code/demo/data/TestAPI.mp4 → D:/ThienPV/code/demo/.trash/TestAPI0.mp4)
[INFO] 17:33:23 - __main__: Running FFmpeg: C:/Users/ADMIN/AppData/Local/Microsoft/WinGet/Packages/Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe/ffmpeg-7.1.1-full_build/bin/ffmpeg.exe -y -i D:/ThienPV/code/demo/data/TestAPI.mp4 -ss 0 -t 10.95 -c:v libx264 -c:a aac D:/ThienPV/code/demo/.trash/TestAPI0.mp4


# Concat

In [9]:
import os
import tempfile

def concat_videos(video_paths: list[str], output_path: str, reencode=False):
    if len(video_paths) < 2:
        raise ValueError("Cần ít nhất 2 video để ghép")

    if reencode:
        # Reencode + filter_complex
        inputs = []
        filter_parts = []
        for idx, path in enumerate(video_paths):
            inputs.extend(["-i", path])
            filter_parts.append(f"[{idx}:v:0][{idx}:a:0]")
        filter_str = "".join(filter_parts) + f"concat=n={len(video_paths)}:v=1:a=1[outv][outa]"
        args = [
            "ffmpeg", "-y",
            *inputs,
            "-filter_complex", filter_str,
            "-map", "[outv]", "-map", "[outa]",
            output_path
        ]
    else:
        # Cách concat bằng danh sách file (nhanh, không reencode)
        with tempfile.NamedTemporaryFile(mode="w+", delete=False, suffix=".txt") as f:
            for path in video_paths:
                f.write(f"file '{os.path.abspath(path)}'\n")
            f.flush()
            list_path = f.name

        args = [
            "ffmpeg", "-y",
            "-f", "concat",
            "-safe", "0",
            "-i", list_path,
            "-c", "copy",
            output_path
        ]

    runf(args)


In [10]:
concat_videos(
    ["D:/ThienPV/code/demo/.trash/TestAPI0.mp4","D:/ThienPV/code/demo/.trash/TestAPI1.mp4"],
    "D:/ThienPV/code/demo/.trash/TestAPI.mp4",
    reencode=True
)


[INFO] 17:33:25 - __main__: Running FFmpeg: C:/Users/ADMIN/AppData/Local/Microsoft/WinGet/Packages/Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe/ffmpeg-7.1.1-full_build/bin/ffmpeg.exe -y -i D:/ThienPV/code/demo/.trash/TestAPI0.mp4 -i D:/ThienPV/code/demo/.trash/TestAPI1.mp4 -filter_complex [0:v:0][0:a:0][1:v:0][1:a:0]concat=n=2:v=1:a=1[outv][outa] -map [outv] -map [outa] D:/ThienPV/code/demo/.trash/TestAPI.mp4


# Remove chroma key

In [11]:
from PIL import Image
import numpy as np
from collections import Counter
import subprocess
import io
import os


def get_dominant_colors_from_image(img: Image.Image, top_n: int = 10):
    """
    Trích xuất top_n màu phổ biến nhất từ ảnh PIL.Image.
    Trả về list tuple: (màu dạng '0xRRGGBB', tỷ lệ phần trăm)
    """
    img = img.convert('RGB')
    img_np = np.array(img)
    pixels = img_np.reshape(-1, 3)

    color_counts = Counter(map(tuple, pixels))
    total = sum(color_counts.values())

    most_common = color_counts.most_common(top_n)

    results = []
    for color, count in most_common:
        hex_color = '0x{:02x}{:02x}{:02x}'.format(*color)
        percent = count / total * 100
        results.append((hex_color, round(percent, 2)))

    return results


def extract_first_frame(video_path: str) -> Image.Image:
    """
    Dùng ffmpeg để trích xuất frame đầu tiên từ video → trả về PIL.Image.
    """
    cmd = [
        FFMPEG_PATH, "-y",
        "-i", video_path,
        "-frames:v", "1",
        "-f", "image2pipe",
        "-vcodec", "png",
        "pipe:1"
    ]
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.DEVNULL)
    return Image.open(io.BytesIO(result.stdout))


def detect_background_color(video_path: str) -> str:
    """
    Dự đoán màu nền của video dựa trên frame đầu tiên.
    Trả về hex string dạng '0xRRGGBB'
    """
    frame = extract_first_frame(video_path)
    colors = get_dominant_colors_from_image(frame, top_n=5)
    return colors[0][0]  # màu phổ biến nhất



In [12]:
import os

def convert_video_format(
    input_path: str,
    output_path: str | None = None,
    codec: str = "qtrle",
    pix_fmt: str = "yuva420p"
):
    """
    Chuyển định dạng video, mặc định từ MP4 sang MOV với hỗ trợ alpha (qtrle).

    Args:
        input_path (str): Đường dẫn video đầu vào.
        output_path (str | None): Đường dẫn đầu ra. Nếu None sẽ đổi đuôi sang .mov tự động.
        codec (str): Codec video, mặc định 'qtrle' để hỗ trợ alpha trong .mov.
        pix_fmt (str): Pixel format, mặc định 'yuva420p' để giữ alpha.
    """
    if output_path is None:
        base, _ = os.path.splitext(input_path)
        output_path = base + ".mov"

    args = [
        "ffmpeg", "-y",
        "-i", input_path,
        "-c:v", codec,
        "-pix_fmt", pix_fmt,
        output_path
    ]

    runf(args)
!

In [13]:
def remove_background(input_path, output_path="temp_fg.webm"):
    # Bước 1: Remove green background (xuất video có alpha)
    cmd_remove_bg = [
        "ffmpeg", "-y", "-i", input_path,
        "-vf", "chromakey=0x25d38d:0.2:0.1,format=yuva420p",
        "-c:v", "libvpx-vp9", "-pix_fmt", "yuva420p", "-auto-alt-ref", "0",
        output_path
    ]
    runf(cmd_remove_bg)


In [14]:
remove_background(
    input_path="D:/ThienPV/code/demo/.trash/TestAPI.mp4",
    output_path="D:/ThienPV/code/demo/.trash/TestAPI_nobg.webm",
)


[INFO] 17:33:45 - __main__: Running FFmpeg: C:/Users/ADMIN/AppData/Local/Microsoft/WinGet/Packages/Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe/ffmpeg-7.1.1-full_build/bin/ffmpeg.exe -y -i D:/ThienPV/code/demo/.trash/TestAPI.mp4 -vf chromakey=0x25d38d:0.2:0.1,format=yuva420p -c:v libvpx-vp9 -pix_fmt yuva420p -auto-alt-ref 0 D:/ThienPV/code/demo/.trash/TestAPI_nobg.webm


# Overlay

In [ ]:
import json
def get_video_resolution(path: str) -> tuple[int, int]:
    """
    Trả về độ phân giải (width, height) của video/image đầu vào.
    """
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}")
    
    cmd = [
        FFPROBE_PATH,
        "-v", "error",
        "-select_streams", "v:0",
        "-show_entries", "stream=width,height",
        "-of", "json",
        path
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)

    try:
        info = json.loads(result.stdout)
        stream = info["streams"][0]
        return int(stream["width"]), int(stream["height"])
    except (KeyError, IndexError, json.JSONDecodeError):
        raise RuntimeError(f"Không thể lấy thông tin độ phân giải từ: {path}\nOutput:\n{result.stdout}")


In [ ]:
def get_video_duration(path: str) -> float:
    """
    Trả về thời lượng (giây) của video/ảnh động.
    """
    cmd = [
        FFPROBE_PATH,
        "-v", "error",
        "-select_streams", "v:0",
        "-show_entries", "format=duration",
        "-of", "default=noprint_wrappers=1:nokey=1",
        path
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    try:
        return float(result.stdout.strip())
    except:
        raise RuntimeError(f"Không lấy được duration từ: {path}")


In [ ]:
import os

def overlay_background(
    fg_path: str,
    bg_path: str,
    output_path: str,
    position: tuple[int, int] = (0, 0),
    scale: float = 1.0,
    bg_loop: bool = True,
    duration: float | None = None,
    format: str = "mp4"
):
    """
    Overlay video nhân vật đã xóa nền (foreground) lên nền (background: video hoặc ảnh).

    Args:
        fg_path (str): Foreground video path (đã xóa nền).
        bg_path (str): Background image or video path.
        output_path (str): Output video path.
        position (tuple[int, int], optional): (x, y) to place foreground.
        scale (float): Tỉ lệ kích thước foreground so với background.
        bg_loop (bool, optional): Lặp lại background nếu ngắn hơn.
        duration (float | None, optional): Duration cố định (nếu không → tự lấy từ foreground).
        format (str, optional): Output format.
    """
    x, y = position
    fg_stream = "[1:v]"
    filter_parts = []

    if scale != 1:
        bg_w, bg_h = get_video_resolution(bg_path)
        w = int(scale * bg_w)
        h = int(scale * bg_h)
        filter_parts.append(f"[1:v]scale={w}:{h}[fg];")
        fg_stream = "[fg]"

    filter_parts.append(f"[0:v]{fg_stream}overlay={x}:{y}[outv]")
    filter_complex = "".join(filter_parts)

    args = ["ffmpeg", "-y"]

    if bg_path.lower().endswith((".jpg", ".jpeg", ".png", ".webp")):
        args.extend(["-loop", "1"])

    args.extend(["-i", bg_path, "-i", fg_path])
    args.extend(["-filter_complex", filter_complex])
    args.extend(["-map", "[outv]", "-map", "1:a?"])

    if duration is None:
        duration = get_video_duration(fg_path)

    args.extend(["-t", str(duration)])
    args.extend(["-c:v", "libx264", "-pix_fmt", "yuv420p", output_path])

    runf(args)


In [ ]:
overlay_background(
    fg_path="D:/ThienPV/code/demo/.trash/TestAPI_nobg.mp4",
    bg_path="D:/ThienPV/code/demo/data/img.jpg",
    output_path="D:/ThienPV/code/demo/.trash/overlay.mp4",
    position=(100, 50),
    scale=0.5,
)


[INFO] 16:00:48 - __main__: Running FFmpeg: C:/Users/ADMIN/AppData/Local/Microsoft/WinGet/Packages/Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe/ffmpeg-7.1.1-full_build/bin/ffmpeg.exe -y -loop 1 -i D:/ThienPV/code/demo/data/img.jpg -i D:/ThienPV/code/demo/.trash/TestAPI_nobg.mp4 -filter_complex [1:v]scale=368:207[fg];[0:v][fg]overlay=100:50[outv] -map [outv] -map 1:a? -t 14.506 -c:v libx264 -pix_fmt yuv420p D:/ThienPV/code/demo/.trash/overlay.mp4


# 